FluTE `$name-tract.dat`, `$name-wf.dat` to my `$name_$size-init.csv`, `$name_$size-flug.dat`. 

*CAVEAT*: Clean the output before committing

Assumed location: `tools`

In [19]:
pwd()

"/Users/yarr/repos/epi-net-m/tools"

FluTE data path: `data/by-tract/flute`; Rel.:`../data/by-tract/flute`

Output path: `data/by-tract`; Rel.:`../data/by-tract`

In [20]:
#reading ingress $name-tract.dat, $name-wf.dat$
using DelimitedFiles
#reading ingress; possibly, output too
using CSV
#transforming the data
using DataFrames
#cool multi-column transforms; mostly for @byrow
#using DataFramesMeta

Start with **initial values**, `$name-tract.dat` -> `$name_$size-init.csv`

In [35]:
flt_IV_suff="-tracts.dat"
my_IV_suff="-init.csv"
ifDir = joinpath("..","data","by-tract","flute")
ofDir = joinpath("..","data","by-tract")
#separates file name fields
fnSep="-"
#separates elements of file names
fnSubSep="_"
#FIPS codes are (State,County,Tract); I'd write them $State$County$Tract
#I'd just concatenate it all; it's SSCCCTTTTTT (at least in seattle-tracts.dat)
#2-digit State, 3-digit County, and 6-digit Tract, and a sensible sort as integers

"_"

In [22]:
#pick only those inding in "-tracts.dat", sort of IVs, with tract ID, pop, long, lat
ivFiles=filter(s -> endswith(s,flt_IV_suff),readdir(ifDir))
#just the names, for later usage
instNames = map(s->split(s,fnSep)[1],ivFiles)

4-element Array{SubString{String},1}:
 "la"
 "one"
 "seattle"
 "usa"

In [36]:
#start with Seattle, #3 in ivFiles
#it doesn't have a header
#the columns are (FIPS_state,FIPS_county,FIPS_tract,pop,long,lat)
dfRaw=CSV.File(joinpath(ifDir,ivFiles[3])
    ,header=false
    ,types=[String,String,String,Int64,Float64,Float64]) |> DataFrame
# nrow(dfRaw) 124 tracts in Seattle
dfRaw[1:5,:] #lazy output truncation

,Column1,Column2,Column3,Column4,Column5,Column6
,String,String,String,Int64,Float64,Float64
1,53,033,000100,5530,47.7275,-122.291
2,53,033,000200,7345,47.7267,-122.309
3,53,033,000300,2485,47.7307,-122.336
4,53,033,000401,4734,47.7282,-122.353
5,53,033,000402,4582,47.7177,-122.354


In [38]:
#copy dfRaw, to prevent mutating it
dfA=DataFrame(dfRaw)#, :Column4 => :N_i, :Column5 =>Long, :Column6 => Lat)
#concatenate the FIPS columns into a new one, “id”
dfA.id = map((s,z,w)-> join([s,z,w]),dfA.Column1,dfA.Column2,dfA.Column3)
#chuck the now-useless cols 1–3
dfB=select(dfA, Not([:1,:2,:3]))

,Column4,Column5,Column6,id
,Int64,Float64,Float64,String
1,5530,47.7275,-122.291,53033000100
2,7345,47.7267,-122.309,53033000200
3,2485,47.7307,-122.336,53033000300
4,4734,47.7282,-122.353,53033000401
5,4582,47.7177,-122.354,53033000402
6,3296,47.7207,-122.367,53033000500
7,7361,47.7171,-122.328,53033000600
8,4189,47.7161,-122.3,53033000700
9,2600,47.7116,-122.286,53033000800


In [33]:
dfA

,Column1,Column2,Column3,Column4,Column5,Column6,id
,String,String,String,Int64,Float64,Float64,String
1,53,033,000100,5530,47.7275,-122.291,53033000100
2,53,033,000200,7345,47.7267,-122.309,53033000200
3,53,033,000300,2485,47.7307,-122.336,53033000300
4,53,033,000401,4734,47.7282,-122.353,53033000401
5,53,033,000402,4582,47.7177,-122.354,53033000402
6,53,033,000500,3296,47.7207,-122.367,53033000500
7,53,033,000600,7361,47.7171,-122.328,53033000600
8,53,033,000700,4189,47.7161,-122.3,53033000700
9,53,033,000800,2600,47.7116,-122.286,53033000800
